In [12]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [13]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [27]:
# Setting these as global variables so that they can be shuffled to ensure an even distribution- many times I wasn't getting all 24/720 labels in the validation set

images = np.load('75/images.npy')
images = images.astype('float32')

# Function that shuffles the images and labels into training, validation, and test sets
def initalize():
    global imgs, train_imgs, val_imgs, test_imgs, train_labels, val_labels, test_labels, distributed
    indices = np.random.permutation(images.shape[0])
    imgs = images[indices]

    split_1 = int(18000*0.8)
    split_2 = int(18000*0.9)

    train_imgs = imgs[:split_1]
    val_imgs = imgs[split_1:split_2]
    test_imgs = imgs[split_2:]

    # Normalizing the images
    train_imgs = train_imgs / 255.0
    test_imgs = test_imgs / 255.0
    val_imgs = val_imgs / 255.0

    labels = np.load('75/labels.npy')
    labels = labels.astype('int32')
    labels = labels[indices]
    train_labels = labels[:split_1]
    val_labels = labels[split_1:split_2]
    test_labels = labels[split_2:]

    train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
    val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
    test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))


In [28]:
initalize()
train_imgs

array([[[[0.3529412 ],
         [0.3529412 ],
         [0.3372549 ],
         ...,
         [0.4627451 ],
         [0.5411765 ],
         [0.47058824]],

        [[0.39215687],
         [0.42352942],
         [0.3882353 ],
         ...,
         [0.49803922],
         [0.49803922],
         [0.47843137]],

        [[0.34901962],
         [0.36078432],
         [0.34509805],
         ...,
         [0.49411765],
         [0.47843137],
         [0.47058824]],

        ...,

        [[0.49019608],
         [0.5176471 ],
         [0.5137255 ],
         ...,
         [0.6039216 ],
         [0.6156863 ],
         [0.58431375]],

        [[0.41568628],
         [0.43529412],
         [0.39607844],
         ...,
         [0.47843137],
         [0.4862745 ],
         [0.4745098 ]],

        [[0.5294118 ],
         [0.5058824 ],
         [0.53333336],
         ...,
         [0.47058824],
         [0.47843137],
         [0.5137255 ]]],


       [[[0.58431375],
         [0.5882353 ],
         [0.59

## Classification Model

In [ ]:
# Convert the time into 24 separate labels
def conv_time_24(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

# Convert the time into 720 separate labels 
def conv_time_720(time):
    return time[0]*60 + time[1]

conv_time = conv_time_24
while True:
    initalize()
    train_labels_converted = np.array([conv_time(time) for time in train_labels])
    test_labels_converted = np.array([conv_time(time) for time in test_labels])
    val_labels_converted = np.array([conv_time(time) for time in val_labels])

    encoder = LabelEncoder()
    test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
    train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
    val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

    OHencoder = OneHotEncoder(sparse_output=False)
    train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
    val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))

    # Check if all labels are present in the validation set, if not, reshuffle
    try:
        val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 24))
        break
    except:
        pass

In [ ]:
def common_sense_error(true, pred):
    true = K.cast(true, 'float32')
    diff1 = 

In [30]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
# First layer - one neuron for each pixel
model.add(keras.layers.Dense(units=5625, activation="relu"))
model.add(keras.layers.Dense(units=1125, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=24, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

print(train_imgs.shape, train_labels_oh.shape)


model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


(14400, 75, 75, 1) (14400, 24)
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.0373 - loss: 3.1781 - val_accuracy: 0.0344 - val_loss: 3.1782
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.0454 - loss: 3.1780 - val_accuracy: 0.0344 - val_loss: 3.1785
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.0446 - loss: 3.1777 - val_accuracy: 0.0344 - val_loss: 3.1787
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.0447 - loss: 3.1775 - val_accuracy: 0.0344 - val_loss: 3.1793
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.0487 - loss: 3.1767 - val_accuracy: 0.0406 - val_loss: 3.1775
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.0553 - loss: 3.1687 - val_accuracy: 0.0550 - val_loss: 3.1381
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.0706 - loss: 3.0941 - val_accuracy: 0.0733 - val_loss: 2.9684
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.0985 - loss: 2.

In [48]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)
accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
4.277777777777778 %


## Regression Model

In [49]:
import tensorflow.keras.backend as K

#returning the common sense difference between two times
def common_sense_error(true, pred):
    true = K.cast(true, 'float32')
    diff_1 = K.abs(true - pred)
    diff_2 = K.abs(true - (pred + 12))

    return K.minimum(diff_1, diff_2)

In [ ]:
initalize()

def conv_time(time):
    return round(time[0] + time[1]/60, 3)

train_labels_reg = np.array([conv_time(time) for time in train_labels])
test_labels_reg = np.array([conv_time(time) for time in test_labels])
val_labels_reg = np.array([conv_time(time) for time in val_labels])

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), input_shape=(75, 75, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=5625, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation="softplus"))
model.compile(loss="mse", optimizer="adam", metrics=[common_sense_error])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, train_labels_reg, epochs=40, batch_size = 512, validation_data = (val_imgs, val_labels_reg), callbacks = [early_stop])

Epoch 1/40


/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/29 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - common_sense_error: 3.7582 - loss: 37.7162

KeyboardInterrupt: 

In [36]:
reg_preds = model.predict(test_imgs)
# for i in range(len(reg_preds)):
#     print(reg_preds[i], test_labels_reg[i])
accuracy = np.mean(abs(reg_preds - test_labels_reg) < 0.01)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.1696604938271605 %


## Two-Headed Model

In [37]:
train_hours = train_labels[:, 0]
train_minutes = train_labels[:, 1]

val_hours = val_labels[:, 0]
val_minutes = val_labels[:, 1]

test_hours = test_labels[:, 0]
test_minutes = test_labels[:, 1]

In [38]:
inp = keras.layers.Input(shape = (75,75,1))
model = keras.layers.Convolution2D(32,kernel_size = (5,5), strides= (2,2), activation = "relu")(inp)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(64,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(64,kernel_size = (1,1),activation = "relu")(model)
model = keras.layers.Flatten()(model)

d = keras.layers.Dense(256,activation = "relu")(model)
d = keras.layers.Dense(256,activation = "relu")(d)
d = keras.layers.Dropout(0.1)(d)
d = keras.layers.Dense(256,activation = "relu")(d)

hour = keras.layers.Dense(256,activation = "relu")(d)
hour = keras.layers.Dense(128,activation = "relu")(hour)
hour = keras.layers.Dense(64,activation = "relu")(hour)
hour = keras.layers.Dense(32,activation = "relu")(hour)
hour = keras.layers.Dense(16,activation = "relu")(hour)
hour = keras.layers.Dense(12,activation= "softmax", name= "hour")(hour)

minute = keras.layers.Dense(256,activation = "relu")(d)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(128,activation = "relu")(minute)
minute = keras.layers.Dense(64,activation = "relu")(minute)
minute = keras.layers.Dense(32,activation = "relu")(minute)
minute = keras.layers.Dense(16,activation = "relu")(minute)
minute = keras.layers.Dense(1, activation = "softplus", name = "minute")(minute)

model = tf.keras.models.Model(inputs=inp, outputs=[hour, minute])
optim = tf.keras.optimizers.Adam()
model.compile(loss=['sparse_categorical_crossentropy', 'mse'], optimizer=optim, metrics=['accuracy',"mae"])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, [train_hours, train_minutes], epochs=30, batch_size = 512, validation_data = (val_imgs, [val_hours, val_minutes]), callbacks = [early_stop])

Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - hour_accuracy: 0.0841 - hour_loss: 2.4895 - loss: 829.4688 - minute_loss: 826.4379 - minute_mae: 23.7829 - val_hour_accuracy: 0.0750 - val_hour_loss: 2.4848 - val_loss: 378.0327 - val_minute_loss: 376.9552 - val_minute_mae: 16.3143
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - hour_accuracy: 0.0844 - hour_loss: 2.4860 - loss: 325.5216 - minute_loss: 323.0148 - minute_mae: 15.3859 - val_hour_accuracy: 0.0789 - val_hour_loss: 2.4858 - val_loss: 307.5529 - val_minute_loss: 305.6083 - val_minute_mae: 15.1643
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - hour_accuracy: 0.0877 - hour_loss: 2.4849 - loss: 302.8543 - minute_loss: 300.3875 - minute_mae: 14.9173 - val_hour_accuracy: 0.0789 - val_hour_loss: 2.4859 - val_loss: 308.9561 - val_minute_loss: 307.1851 - val_minute_mae: 15.1973
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - hour_accuracy: 0.0844 - hour_loss: 2.4850 - loss: 304.8977 - minute_loss: 302.4233 - minute_

In [ ]:
predictions = model.predict(test_imgs)
hour_p = np.argmax(predictions[0], axis = 1)
minutes_p = predictions[1]

accuracy = np.mean(np.abs(hour_p - test_hours) < 1) * np.mean(np.abs(minutes_p - test_minutes) < 5)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1.363423353909465 %


## Label Transformation using Periodic Function 


In [ ]:
sine_time_train = (train_hours*60 + train_minutes) 
sine_time_test = (test_hours*60 + test_minutes)  
sine_time_valid = (val_hours*60 + val_minutes) 

In [ ]:
np.max(sine_time_train)

np.int32(719)

In [ ]:
sine_angle_test = (sine_time_test/720)*2*np.pi
sine_angle_train = (sine_time_train/720)*2*np.pi
sine_angle_valid = (sine_time_valid/720)*2*np.pi

In [ ]:
# Sine Regression CNN

sin_reg = keras.models.Sequential()
sin_reg.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides = (2,2),input_shape=(75, 75, 1)))
sin_reg.add(keras.layers.MaxPooling2D(pool_size=2))
sin_reg.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.MaxPooling2D(pool_size=2))
sin_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


sin_reg.add(keras.layers.Flatten())
sin_reg.add(keras.layers.Dense(units=512, activation='relu'))
sin_reg.add(keras.layers.Dense(units=512, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dropout(0.2))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dropout(0.2))
sin_reg.add(keras.layers.Dense(units=128, activation='relu'))
sin_reg.add(keras.layers.Dense(units=64, activation='relu'))
sin_reg.add(keras.layers.Dense(units=32, activation='relu'))
sin_reg.add(keras.layers.Dense(units=1, activation="softplus"))
early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
sin_reg.compile(loss='mse', optimizer= optimizer, metrics=['mae'])
sin_reg.fit(train_imgs, sine_angle_train, epochs=45, batch_size = 512, validation_data = (val_imgs, sine_angle_valid), callbacks = [early_stop])

Epoch 1/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 0.1435 - mae: 0.2371 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 2/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7896e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 3/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 3.7661e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 4/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7713e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 5/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7717e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 6/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 3.8013e-05 - mae: 0.0056 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 7/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7782e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 8/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7938e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoc

In [ ]:
predictions = sin_reg.predict(test_imgs)

def difference_func(pred,y):
  pred = np.transpose(pred)
  diff_one = np.maximum(pred,y) - np.minimum(pred,y)
  diff_two = np.minimum(pred,y) + 1 - np.maximum(pred,y)
  return np.minimum(diff_one,diff_two)

result = difference_func(predictions,sine_angle_test).reshape(-1)

accuracy = np.mean(result < np.pi/6)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
100.0 %
